In [1]:
from pprint import pprint

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from prepare import basic_clean, lemmatize

What other types of models (i.e. different classifcation algorithms) could you use?


How do the models compare when trained on term frequency data alone, instead of TF-IDF values?

Term Frequency (TF)

Term frequency can be calculated in a number of ways, all of which reflect how frequently a word appears in a document.

Raw Count: This is simply the count of the number of occurances of each word.

Frequency: The number of times each word appears divided by the total number of words.

Augmented Frequency: The frequency of each word divided by the maximum frequency. This can help prevent bias towards larger documents.

In [2]:
document = """What are the Math and Stats Principles You Need for Data Science?
Oct 21, 2020 | Data Science


Coming into our Data Science program, you will need to know some math and stats. However, many of our applicants actually learn in the application process – you don’t need to be an expert before applying! Data science is a very accessible field to anyone dedicated to learning new skills, and we can work with any applicant to help them learn what they need to know. But what “skills” do we mean, e
"""

# clean up the text
document = document.lower().replace(',', '').replace('.', '')
# transform into a series
words = pd.Series(document.split())

# From the Series we can extract the value_counts, which is our raw count
# for term frequency. Once we have the raw counts, we can calculate the
# other measures.
(pd.DataFrame({'raw_count': words.value_counts()})
 .assign(frequency=lambda df: df.raw_count / df.raw_count.sum())
 .assign(augmented_frequency=lambda df: df.frequency / df.frequency.max()))

,raw_count,frequency,augmented_frequency
to,6,0.065934,1.000000
data,4,0.043956,0.666667
need,4,0.043956,0.666667
what,3,0.032967,0.500000
and,3,0.032967,0.500000
...,...,...,...
process,1,0.010989,0.166667
–,1,0.010989,0.166667
don’t,1,0.010989,0.166667
be,1,0.010989,0.166667
